In [ ]:
from copy import deepcopy
import hubbard as hbb
import numpy as np
import scipy.sparse as sp
from qiskit import AncillaRegister, ClassicalRegister, QuantumCircuit
import qmatchatea as qtea
from qmatchatea.py_emulator import QcMps
from qmatchatea.preprocessing import _preprocess_qk

from hubbard.hamiltonian_terms import hopping_hamiltonian
import os

import matplotlib.pyplot as plt

shape = (4, 2)

In [ ]:
def simple_visualizer(shape, state):
    state_str = []
    current_str = "│"
    vertical = " "*2+"│     "*shape[0]
    top = "┌─┴─┐ "*shape[0]
    bottom = "└─┬─┘ "*shape[0]
    for jj in range(shape[1]):
        state_str.append(bottom + "\n")
        for ii in range(shape[0]):
            if ii>0:
                current_str += "┤"
            idx = 2*(ii+ jj*shape[0])
            current_str += state[idx]+","+state[idx+1]+"├─"
        state_str.append(current_str[:-1]+"\n")
        state_str.append(top + "\n")
        if jj < shape[1]-1:
            state_str.append(vertical+"\n")
        current_str = "│"
    
    return "".join(state_str[::-1])

In [ ]:
shape = (4, 2)
num_up = np.prod(shape)//2
num_down = np.prod(shape)//2
max_bond_dim = 10
conv_params = qtea.QCConvergenceParameters(max_bond_dimension=max_bond_dim, singval_mode='C')
all_states = hbb.all_possible_matter_states(shape, num_up, num_down)
print(f"There will be a total of {all_states.shape[0]} states")

# Hamiltonian of the symmetry sector
#symmetric_hamiltonian = np.zeros([all_states.shape[0]]*2, dtype=complex)
symmetric_hamiltonian = sp.lil_matrix(tuple([all_states.shape[0]]*2), dtype=complex)

# Vertexes definition
vertexes = [(ii, jj) for jj in range(shape[1]) for ii in range(shape[0])]
sites = [f"q({ii}, {jj})" for jj in range(shape[1]) for ii in range(shape[0])]
site_ordering = []
idx = 0
for ii, jj in vertexes:
    if (ii+jj)%2==0:
        site_ordering += [idx, idx+1]
    else:
        site_ordering += [idx+1, idx]
    idx += 2
# Plaquettes definition
plaquettes = [(ii, jj) for jj in range(shape[1]-1) for ii in range(shape[0]-1) ]
# Number of link qubits
num_links = shape[0]*(shape[1]-1) + shape[1]*(shape[0]-1)
num_qubs = 2*len(vertexes) + num_links + 1
qancilla = AncillaRegister(1, 'a0')
cancillas = [ClassicalRegister(1, f'ca{ii}') for ii in range(len(plaquettes)+len(vertexes)+1)]


# ============= Initialize Hubbard circuit =============
regs, qc = hbb.hubbard_circuit(shape, qancilla, cancillas )

if False:
    for ii, vv in enumerate(vertexes):
        qc = hbb.apply_vertex_parity_stabilizer(qc, regs, qancilla, cancillas[len(plaquettes)+ii], vv)
    for ii, pp in enumerate(plaquettes):
        qc = hbb.apply_plaquette_stabilizers(qc, regs, qancilla[0], cancillas[ii], pp )

    lin_qc = _preprocess_qk(qc)

    # Generate all the correct MPS states that respect
    # all the stabilizer checks
    mps_states = []
    for state in all_states:
        temp_qc = QuantumCircuit(*qc.qregs, *qc.cregs)
        temp_state = QcMps(num_qubs, conv_params)
        for idx, qub_state in enumerate(state):
            if qub_state == 1:
                site = sites[ idx//2 ]
                specie = "u" if idx%2 == 0 else "d"
                temp_qc.x( regs[site][specie] )
        
        temp_state.run_from_qk(temp_qc)
        temp_state.run_from_qk(lin_qc)
        mps_states.append(temp_state)

In [ ]:
save_dir = f"data/exact/{shape[0]}x{shape[1]}"
eigenvalues = np.loadtxt(os.path.join(save_dir, "eigenvalues.txt"))
eigenvectors = np.loadtxt(os.path.join(save_dir, "eigenvectors.txt"), dtype=complex)

eigenvalues_jw = np.loadtxt(os.path.join(save_dir, "eigenvalues_jw.txt"))
eigenvectors_jw = np.loadtxt(os.path.join(save_dir, "eigenvectors_jw.txt"), dtype=complex)

In [ ]:
plt.plot(np.abs(eigenvalues-eigenvalues_jw), 'o--' )

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

numbering = np.arange(len(eigenvalues))
first_band = len(eigenvalues[eigenvalues < 0.5])
last_band = len(eigenvalues[eigenvalues > 1.5])
middle_band = len(eigenvalues) - first_band - last_band


ax.plot(numbering[eigenvalues < 0.5], eigenvalues[eigenvalues < 0.5],
    '.--', color="navy", label=f"{first_band} states")

condition = np.logical_and(eigenvalues > 0.5,  eigenvalues<1.5)
ax.plot(numbering[condition], eigenvalues[condition],
    '.--', color="forestgreen", label=f"{middle_band} states")

ax.plot(numbering[eigenvalues > 1.5], eigenvalues[eigenvalues > 1.5],
    '.--', color="firebrick", label=f"{last_band} states")


ax.set_xlabel("Eigenvalue index", fontsize=14)
ax.set_ylabel("Energy", fontsize=14)
ax.set_title(f"Lattice ({shape[0]}x{shape[1]}), U=1, t=0.1", fontsize=16)
ax.grid()
ax.legend(fontsize=14)

plt.savefig("spectrum.pdf")

fig, ax = plt.subplots(figsize=(6, 4))

numbering = np.arange(len(eigenvalues_jw))
first_band = len(eigenvalues_jw[np.arange(len(eigenvalues_jw))<420])
last_band = len(eigenvalues_jw[eigenvalues_jw > 1.7])
middle_band = len(eigenvalues_jw) - first_band - last_band


ax.plot(numbering[np.arange(len(eigenvalues_jw))<420], eigenvalues_jw[np.arange(len(eigenvalues_jw))<420],
    '.--', color="navy", label=f"{first_band} states")

condition = np.logical_and(np.arange(len(eigenvalues_jw))>420,  eigenvalues_jw<1.7)
ax.plot(numbering[condition], eigenvalues_jw[condition],
    '.--', color="forestgreen", label=f"{middle_band} states")

ax.plot(numbering[eigenvalues_jw > 1.7], eigenvalues_jw[eigenvalues_jw > 1.7],
    '.--', color="firebrick", label=f"{last_band} states")


ax.set_xlabel("Eigenvalue index", fontsize=14)
ax.set_ylabel("Energy", fontsize=14)
ax.set_title(f"Lattice ({shape[0]}x{shape[1]}), U=1, t=0.1", fontsize=16)
ax.grid()
ax.legend(fontsize=14)

plt.savefig("spectrum_jw.pdf")

In [ ]:
plt.plot(eigenvalues_jw[:10], '.--')

In [ ]:
all_state_strings = ["".join(list(state)) for state in all_states.astype(str) ]

In [ ]:
def get_eigenvector_dict(idx, eigenvectors, str_states):
    eigv_dict = {}
    for jdx, coeff in enumerate(eigenvectors[:, idx]):
        if np.isclose(np.abs(coeff), 0):
            continue
        else:
            eigv_dict[str_states[jdx]] = coeff
    
    return eigv_dict

def get_eigenvector_str(idx, eigenvectors, str_states):
    eigv_str = ""
    for jdx, coeff in enumerate(eigenvectors[:, idx]):
        if np.isclose(np.abs(coeff), 0):
            continue
        elif np.isclose(np.imag(coeff), 0):
            coeff = np.real(coeff)
            eigv_str += "%.3f"%(coeff)
        elif np.isclose(np.real(coeff), 0):
            coeff = np.imag(coeff)
            eigv_str += "%.3f j"%(coeff)
        else:
            eigv_str += "(%.3f+ %.3f j)"%(np.real(coeff), np.imag(coeff))
        
        eigv_str += f"|{str_states[jdx]}> + "
    
    
    return eigv_str[:-3]

In [ ]:
get_eigenvector_str(-1, eigenvectors, all_state_strings)

In [ ]:
idx = 1

eigv_dict = get_eigenvector_dict(idx, eigenvectors, all_state_strings)

values = np.array(list(eigv_dict.values()))
values = values*np.conj(values)
ordering = np.argsort(values)[::-1]
values = values[ordering]
states = np.array(list(eigv_dict.keys()))[ordering]

with open(f"eigenstate_{idx}.txt", "w") as fh:
    for key, val in zip(states, values):   
        fh.write(str(np.real(val))+"\n" )
        fh.write( simple_visualizer(shape, key) )
        fh.write("\n\n")

In [ ]:
idx = 0

eigv_dict = get_eigenvector_dict(idx, eigenvectors_jw, all_state_strings)

values = np.array(list(eigv_dict.values()))
values = values*np.conj(values)
ordering = np.argsort(values)[::-1]
values = values[ordering]
states = np.array(list(eigv_dict.keys()))[ordering]

repetitions = []
probabilities = []
total_states = []

jdx = -1
old_val = 0
for key, val in zip(states, values):   
    val = np.real(val)
    if np.isclose(old_val, val, atol=0.0):
        repetitions[jdx] += 1
    else:
        repetitions.append(1)
        probabilities.append(val)
        total_states.append(key)
        old_val = val
        jdx += 1

with open(f"eigenstate_{idx}.txt", "w") as fh:
    for key, val, prob in zip(total_states, repetitions, probabilities):   
        fh.write(f"{np.real(val)} {prob} \n" )
        fh.write( simple_visualizer(shape, key) )
        fh.write("\n\n")

In [ ]:
def get_populations(state_dict):
    populations = np.zeros(3)
    for key, val in state_dict.items():
        state = list(key)
        count = 0
        for ii in range(0, len(state), 2):
            if "".join( state[ii:ii+2]) == "11":
                count += 1

        if count == 0:
            populations[0] += np.real( val * np.conj(val) )
        elif count == 1:
            populations[1] += np.real( val * np.conj(val) )
        elif count == 2:
            populations[2] += np.real( val * np.conj(val) )
    
    return populations


In [ ]:
populations = []

for ii in range(len(eigenvalues)):
    eigv_dict = get_eigenvector_dict(ii, eigenvectors, all_state_strings)
    populations.append(
        get_populations(eigv_dict)
    )
populations = np.array(populations)

In [ ]:
import matplotlib as mpl
fig, ax0 = plt.subplots(figsize=(12, 4))

ax0.plot(eigenvalues_jw, populations[:, 0],
    '.--', color="navy", label="No double occupancy")
ax0.plot(eigenvalues_jw, populations[:, 1],
    '.--', color="forestgreen", label="One double occupancy")
ax0.plot(eigenvalues_jw, populations[:, 2],
    '.--', color="firebrick", label="Two double occupancies")

ax0.set_xlabel("Eigenvalue energy", fontsize=14)
ax0.set_ylabel("Probability of measuring \n in a configuration", fontsize=14)
ax0.set_title(f"Lattice ({shape[0]}x{shape[1]}), U=1, t=0.1", fontsize=16)
ax0.grid()
ax0.legend(fontsize=12)

plt.savefig("configurations.pdf")

In [ ]:
np.arange(0, len(populations[:, 0]))[ populations[:, 0]-populations[:, 1]<0][0]